# Transformacija

In [10]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
import warnings

warnings.filterwarnings('ignore')

In [11]:
df = pd.read_csv("dataset/cleaned_deliverytime.csv")
df.head()

,Unnamed: 0,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),Datetime_Ordered,Datetime_Picked
0,0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Sunny,High,2,Snack,motorcycle,0,0,Urban,24,2022-03-19 11:30:00,2022-03-19 11:45:00
1,1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Stormy,Jam,2,Snack,scooter,1,0,Metropolitian,33,2022-03-25 19:45:00,2022-03-25 19:50:00
2,2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Sandstorms,Low,0,Drinks,motorcycle,1,0,Urban,26,2022-03-19 08:30:00,2022-03-19 08:45:00
3,3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Sunny,Medium,0,Buffet,motorcycle,1,0,Metropolitian,21,2022-04-05 18:00:00,2022-04-05 18:10:00
4,4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Cloudy,High,1,Snack,scooter,1,0,Metropolitian,30,2022-03-26 13:30:00,2022-03-26 13:45:00


ID podatak nije potreban prema EDA-i, ali se kasnije koristi za određivanje grada u kojem se odvila dostava pa se neće izbaciti.

Geografska širina i dužina su ponekad negativne iako se iz Delivery_person_id može vidjeti da se zapravo lokacija nalazi u Indiji.

In [12]:
df['Restaurant_latitude'] = df.apply(lambda x: abs(x['Restaurant_latitude']), axis=1)
df['Restaurant_longitude'] = df.apply(lambda x: abs(x['Restaurant_longitude']), axis=1)
df['Delivery_location_latitude'] = df.apply(lambda x: abs(x['Delivery_location_latitude']), axis=1)
df['Delivery_location_longitude'] = df.apply(lambda x: abs(x['Delivery_location_longitude']), axis=1)

Tip narudžbe i vozile stavljen je na mala slova zbog konzistentnosti. 

In [13]:
df['Type_of_order'] = df.apply(lambda x: x['Type_of_order'].lower().strip(), axis=1)
df['Type_of_vehicle'] = df.apply(lambda x: x['Type_of_vehicle'].lower().strip(), axis=1)

Sve primjeri s koordinatama izvan Indije treba izbaciti. 

In [14]:
df = df[(df['Delivery_location_latitude'] > 8) & (df['Delivery_location_latitude'] < 38) &
        (df['Delivery_location_longitude'] > 68) & (df['Delivery_location_longitude'] < 98) &
        (df['Restaurant_latitude'] > 8) & (df['Restaurant_latitude'] < 38) &
        (df['Restaurant_longitude'] > 68) & (df['Restaurant_longitude'] < 98)]



In [15]:
df['distance(km)'] = df.apply(lambda x: geodesic((x['Restaurant_latitude'], x['Restaurant_longitude']), (x['Delivery_location_latitude'], x['Delivery_location_longitude'])).km, axis=1)

In [16]:
pd.DataFrame.to_csv(df, "dataset/transformed_deliverytime.csv", index=False)